In [1]:

from src.core import metrics

import yfinance as yf

prices = yf.download("SPY, AGG")["Adj Close"]

start = metrics.to_starts(prices)
end = metrics.to_ends(prices)
end - start

[*********************100%***********************]  2 of 2 completed


AGG    7119 days
SPY   11014 days
dtype: timedelta64[ns]

In [ ]:
import glob
from datetime import date, timedelta
import numpy as np
import pandas as pd

path = "price/*.xls"
files = glob.glob(pathname=path)

yesterday = date.today() - timedelta(days=1)

for file in files:
    data = pd.read_excel(file)[["종목", "현재가"]]
    data.columns = ["ticker", "value"]
    if pd.api.types.is_object_dtype(data["value"].dtype):
        data["value"] = data["value"].str.replace(",", "").astype(float)
    data["date"] = yesterday
    data["field"] = "CLOSE"
    print(data)


In [ ]:
from src.core import strategy
from src.core import metrics
import yfinance as yf

prices = yf.download("XLU, XLK, XLB, XLP, XLY, XLI, XLV, XLF")["Adj Close"]

strategy = strategy.Momentum(prices).simulate(
    start="2015-1-1", min_volatility=0.12
)

strategy.value.plot()
print(strategy.analytics())
